# Lightweight Development Pipelines with DVC

In this notebook we will highlight important elements of DVC. You can find extensive information for dvc on their [website](https://dvc.org).

As a showcase we will implement a simple classification pipeline.

### Some Preparations

In [ ]:
# --no-scm because we don't want to interfere with the workshops' git
!dvc init -f --no-scm

Optional: We add a new remote storage (could be S3, GCS, SSH, ...)

In [ ]:
!dvc remote add -d -f local_storage /tmp/dvc_introduction

Let's check our current status. Attention: DVC does not have a sophisticated git-like `stage area`, but a cache-directory, that is being synced with the remote.

In [ ]:
!dvc status -c

That wasn't too surprising...

We can either add files to our DVC versioning by manually adding them or implicitly in a pipeline.

### Building a Pipeline

In [ ]:
%%sh 
dvc run -f configure.dvc \
        -d DVC_task.py \
        -o output-task/config.pickle \
        python DVC_task.py configure output-task/config.pickle

In [ ]:
%%sh 
dvc run -f train.dvc \
        -d DVC_task.py \
        -d output-task/config.pickle \
        -d /keras2production/fruits \
        -o output-task/model.h5 \
        python DVC_task.py train_model /keras2production/fruits output-task/config.pickle output-task/model.h5

In [ ]:
%%sh 
dvc run -f Dvcfile \
        -d DVC_task.py \
        -d output-task/model.h5 \
        -O ../12-models/fruits/2 \
        python DVC_task.py export output-task/model.h5 ../12-models/fruits/2

### Inspecting and Modifying a Pipeline 

In [ ]:
%%sh 
dvc pipeline show --ascii

In [ ]:
!dvc status -c

In [ ]:
!dvc push

In [ ]:
!dvc repro

Let's modify a file and reproduce our pipeline!

#### New Features

Get a file from another (external) git+DVC repository.

In [ ]:
!dvc get https://github.com/iterative/example-get-started model.pkl

In [ ]:
!rm model.pkl

Get a file *including* its .dvc file from another (external) git+DVC repository.

In [ ]:
!dvc import https://github.com/iterative/example-get-started model.pkl

#### Debug only - pls ignore :-)

In [114]:
%%sh
rm -rf .dvc
rm -rf *.dvc
rm Dvcfile
rm -rf /tmp/dvc_introduction